In [5]:
# 查看本地镜像
docker images pycon/webapp

REPOSITORY          TAG                 IMAGE ID            CREATED             VIRTUAL SIZE
pycon/webapp        latest              4d6e15dedf9e        15 hours ago        817.6 MB


In [9]:
# 创建 webapp 容器，并运行该容器
docker run -d --name webapp pycon/webapp

In [1]:
# 查看运行中的容器
docker ps

CONTAINER ID        IMAGE               COMMAND                CREATED             STATUS              PORTS               NAMES
1b79abd1f204        pycon/webapp        "docker-entrypoint.s   18 seconds ago      Up 18 seconds       80/tcp              webapp              


In [5]:
# 查看 webapp 容器的 IP 地址
docker inspect webapp | grep "IPAddress"

        "IPAddress": "172.17.0.34",
        "SecondaryIPAddresses": null,


访问 webapp 服务

In [3]:
docker stop webapp

webapp


In [4]:
docker start webapp

webapp


In [6]:
# pull bind9 image
# docker pull somejedi/docker-bind9
docker images somejedi/docker-bind9

REPOSITORY              TAG                 IMAGE ID            CREATED             VIRTUAL SIZE
somejedi/docker-bind9   latest              7c1c3d63d711        13 months ago       295.6 MB


In [7]:
# start bind9 DNS server
# cd /mnt/workspace/
docker run -d \
-p 172.17.42.1:53:53/udp \
--name dns \
-v /mnt/workspace/pycon-ddns-docker/bind:/data somejedi/docker-bind9 \
/usr/sbin/named -c /etc/bind/named.conf -g

939db8a7ba5d8399be7ce30d559fe905cf53676aa94d6efe67b1b25438a0ffa8


在 Ubuntu 中，设置本地 DNS
```
# /etc/resolv.conf
nameserver 172.17.42.1
nameserver 127.0.1.1
```

In [8]:
docker exec dns cp /etc/bind/db.local /var/lib/bind/db.devops.it
docker restart dns

dns


In [3]:
docker start dns

Error response from daemon: no such id: dns
Error: failed to start containers: [dns]


In [9]:
docker logs dns

12-Sep-2015 08:28:59.504 starting BIND 9.9.5-3-Ubuntu -c /etc/bind/named.conf -g
12-Sep-2015 08:28:59.504 built with '--prefix=/usr' '--mandir=/usr/share/man' '--infodir=/usr/share/info' '--sysconfdir=/etc/bind' '--localstatedir=/var' '--enable-threads' '--enable-largefile' '--with-libtool' '--enable-shared' '--enable-static' '--with-openssl=/usr' '--with-gssapi=/usr' '--with-gnu-ld' '--with-geoip=/usr' '--with-atf=no' '--enable-ipv6' '--enable-rrl' '--enable-filter-aaaa' 'CFLAGS=-fno-strict-aliasing -DDIG_SIGCHASE -O2'
12-Sep-2015 08:28:59.504 ----------------------------------------------------
12-Sep-2015 08:28:59.504 BIND 9 is maintained by Internet Systems Consortium,
12-Sep-2015 08:28:59.504 Inc. (ISC), a non-profit 501(c)(3) public-benefit 
12-Sep-2015 08:28:59.504 corporation.  Support and training for BIND 9 are 
12-Sep-2015 08:28:59.504 available at https://www.isc.org/support
12-Sep-2015 08:28:59.504 ----------------------------------------------------
12-Sep-2015 08:28:59.5

In [14]:
# lookup domain webapp.devops.it
nslookup webapp.devops.it 172.17.42.1

Server:		172.17.42.1
Address:	172.17.42.1#53

Name:	webapp.devops.it
Address: 172.17.0.37



In [11]:
# update domain webapp.devops.it
echo "server 172.17.42.1
zone devops.it
update add webapp.devops.it 600 A 172.17.0.34
send" | nsupdate -k /mnt/workspace/pycon-ddns-docker/ddns/Kdevops.it.+157+62009.key

In [13]:
# 重启 webapp 容器，演示 DDNS
docker restart webapp

webapp


清理演示环境

In [19]:
docker ps

0


In [ ]:
docker stop webapp

In [ ]:
docker rm webapp

In [3]:
docker stop dns

Error response from daemon: no such id: dns
Error: failed to remove containers: [dns]


In [ ]:
docker rm dns